In [29]:
#import the libraries

from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

Create Model

In [23]:
#old model
model = keras.Sequential([
  keras.layers.Rescaling(1./255),
  keras.layers.Conv2D(32, (3,3), activation="relu"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(64, (3,3), activation="relu"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(128, (3,3), activation="relu"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(128, (3,3), activation="relu"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(256, (3,3), activation="relu"),
  keras.layers.Flatten(),
  keras.layers.Dense(1, activation="sigmoid"),

  
])

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

DATA LOADING

In [24]:
#Load the data
definitions = pd.read_csv('OPTED-Dictionary.csv')
simpDef1 = pd.read_excel('ChildFriendlyDefinitions.xlsx', sheet_name='Sheet1')
simpDef2 = pd.read_json('data.json')


ACTUAL DEFINITIONS

In [21]:
filteredDefs = definitions.drop(['Count', 'POS'], axis=1)
filteredDefs

SIMPLIFIED DEFINITIONS

Part A

In [25]:
#Filter simpDef1
simpDef1_Filter = simpDef1.drop(['Exemplar'], axis=1)

In [26]:
#Filter simpDef2
simpDef1_Filter

,Word,Child Friendly Definition
0,Accessible,When something is accessible it means anyone c...
1,Accommodate,You accommodate when you change something that...
2,Accomplish,"If you accomplish something, you succeed in do..."
3,Achieve,"If you achieve something, you succeed in doing..."
4,Acre,An acre is a very large area of land about the...
...,...,...
164,Value,The value of a place or thing is how much mone...
165,Verify,"If you verify something, you make sure that it..."
166,Vigilant,Someone who is vigilant pays careful attention...
167,Visible,"When something is visible, you can see it."


Part B

In [27]:
simpDef2_filter = simpDef2

In [28]:
simpDef2_filter

,word,definition
0,a1,the first letter of the English alphabet. [3 d...
1,a2,any one of a group or kind. [3 definitions]
2,a3,in every or in each; for every or for each.
3,a-1,"a prefix that means ""on,"" ""at,"" ""into,"" or ""to."""
4,a-2,"a prefix that means ""away,"" ""from,"" or ""off."" ..."
...,...,...
15123,zoo,"a place where living animals, especially wild ..."
15124,zoo-,"a prefix that means ""animal."""
15125,zoology,the science and study of animals.
15126,zoom,to move quickly while making a low humming sou...


Getting Dataset ready for training

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Training the Algorithm

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Predicting/Plotting Data

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()